<a href="https://colab.research.google.com/github/thaitruong018/Web_Scraping_Selenium_Python/blob/main/Tiki_Scraping_Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Problem Statement


**Overview**: Build a web-crawler that take in a Tiki URL and return a dataframe  
**Requirements** 
The final dataframe should contain the following informations:  
    - Product Name  
    - Price  
    - URL of the product image  
    - URL of that product page


#Install resources

In [ ]:
# install selenium and other resources for crawling data
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad

In [ ]:
### Import libraries
import re
import time
import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

#Configuration for Driver and links

In [ ]:
###############
### GLOBALS ###
###############

# Header for chromedriver
HEADER = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36'}
# Urls
TIKI                = 'https://tiki.vn'
MAIN_CATEGORIES = [
    {'Name': 'Điện Thoại - Máy Tính Bảng',
     'URL': 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Tử - Điện Lạnh',
     'URL': 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner'},

    {'Name': 'Phụ Kiện - Thiết Bị Số', 
     'URL': 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner'},

    {'Name': 'Laptop - Thiết bị IT', 
     'URL': 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner'},

    {'Name': 'Máy Ảnh - Quay Phim', 
     'URL': 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner'},

    {'Name': 'Điện Gia Dụng', 
     'URL': 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner'},

    {'Name': 'Nhà Cửa Đời Sống', 
     'URL': 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng Tiêu Dùng - Thực Phẩm', 
     'URL': 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner'},

    {'Name': 'Đồ chơi, Mẹ & Bé', 
     'URL': 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner'},

    {'Name': 'Làm Đẹp - Sức Khỏe', 
     'URL': 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner'},

    {'Name': 'Thể Thao - Dã Ngoại', 
     'URL': 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner'},

    {'Name': 'Xe Máy, Ô tô, Xe Đạp', 
     'URL': 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_banner'},

    {'Name': 'Hàng quốc tế', 
     'URL': 'https://tiki.vn/hang-quoc-te/c17166?src=c.17166.hamburger_menu_fly_out_banner'},

    {'Name': 'Sách, VPP & Quà Tặng', 
     'URL': 'https://tiki.vn/nha-sach-tiki/c8322?src=c.8322.hamburger_menu_fly_out_banner'},

    {'Name': 'Voucher - Dịch Vụ - Thẻ Cào', 
     'URL': 'https://tiki.vn/voucher-dich-vu/c11312?src=c.11312.hamburger_menu_fly_out_banner'}
]

# Global driver to use throughout the script
DRIVER = None

#Function to Start and Close Driver

In [ ]:
# Function to (re)start driver
def start_driver(force_restart=False):
    global DRIVER
    
    if DRIVER is not None:
        if force_restart:
            DRIVER.close()
        else:
            raise RuntimeError('ERROR: cannot overwrite an active driver. Please close the driver before restarting.')
    
    # Setting up the driver
    options = webdriver.ChromeOptions()
    options.add_argument('-headless') # we don't want a chrome browser opens, so it will run in the background
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')

    DRIVER = webdriver.Chrome('chromedriver',options=options)

# Wrapper to close driver if its created
def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
    DRIVER = None

In [ ]:
close_driver()

In [ ]:
start_driver()

In [ ]:
# count how many product item
all_product_items = DRIVER.find_elements_by_class_name("product-item")
len(all_product_items)

64

In [ ]:
product_item = all_product_items[1]
product_item

<selenium.webdriver.remote.webelement.WebElement (session="f68d53108c1433ca0d08d3d3efaabd81", element="f774b18b-7ef7-468f-8a81-c1596c6d6e40")>

In [ ]:
# TESTING

# Find the name of a item
class_name = product_item.find_element_by_class_name("name")
tag_span = class_name.find_element_by_tag_name("span")
item_name = tag_span.get_attribute("innerHTML")
#print(item_name)

# Find the price of a item
class_price_discount__price = product_item.find_element_by_class_name("price-discount__price")
a= class_price_discount__price.get_attribute("innerHTML")
b = a.strip(" đ")
print(b)
#print(class_price_discount__price.get_attribute("innerHTML"))
'''
# Find the link of a item
#product_id = DRIVER.find_element_by_class_name("ProductList__Wrapper-sc-1dl80l2-0 Kxajl")
#tag_name_a= DRIVER.find_element_by_tag_name("a") #DRIVER.find_element_by_tag_name("a")
product_link = product_item.get_attribute('href')
print(product_link)

# Find the image of a item
class_thumb_nail = product_item.find_element_by_class_name("thumbnail")
tag_img = class_thumb_nail.find_elements_by_tag_name("img")
product_img= tag_img[-1].get_attribute("src")
print(product_img)
# if len(tag_img) ==2:
#   product_img = tag_img[-1].get_attribute("src")
#   #print(product_img)
# else:
#   product_img = tag_img[0].get_attribute("src")
#   #print(product_img)
'''



14.290.000 ₫


'\n# Find the link of a item\n#product_id = DRIVER.find_element_by_class_name("ProductList__Wrapper-sc-1dl80l2-0 Kxajl")\n#tag_name_a= DRIVER.find_element_by_tag_name("a") #DRIVER.find_element_by_tag_name("a")\nproduct_link = product_item.get_attribute(\'href\')\nprint(product_link)\n\n# Find the image of a item\nclass_thumb_nail = product_item.find_element_by_class_name("thumbnail")\ntag_img = class_thumb_nail.find_elements_by_tag_name("img")\nproduct_img= tag_img[-1].get_attribute("src")\nprint(product_img)\n# if len(tag_img) ==2:\n#   product_img = tag_img[-1].get_attribute("src")\n#   #print(product_img)\n# else:\n#   product_img = tag_img[0].get_attribute("src")\n#   #print(product_img)\n'

#Function to get info from one product

In [ ]:
#################
### FUNCTIONS ###
#################


# Function to extract product info from the product
def get_product_info_single(product_item):
    #print(product_item.get_attribute("outerHTML"))
    d = {'name':'',
         'price':'',
         'product_url':'',
         'image':''}

    # name get name through find_element_by_class_name
    try:
        
        class_name = product_item.find_element_by_class_name("name")
        tag_span = class_name.find_element_by_tag_name("span")
        item_name = tag_span.get_attribute("innerHTML")
        d['name'] = item_name  

    except NoSuchElementException:
        pass

    # get price find_element_by_class_name
    try:
        
        class_price_discount__price = product_item.find_element_by_class_name("price-discount__price")
        # current price  ??????????????
        d['price'] = class_price_discount__price.get_attribute("innerHTML") 
    except NoSuchElementException:
        d['price'] = -1
    
    # get link from .get_attribute()
    try:
        
        #CLEAN LINK WITH STRING MANUPULATION
        d['product_url']= product_item.get_attribute('href')
        
    except NoSuchElementException:
        pass
    
    # get thumbnail by class_name and Tag name and get_attribute()
    try:
        
        class_thumb_nail = product_item.find_element_by_class_name("thumbnail")
        tag_img = class_thumb_nail.find_elements_by_tag_name("img")
        d['image'] = tag_img[-1].get_attribute("src")
        # if len(tag_img) ==2:
        #   d['image'] = tag_img[1].get_attribute("src")
        # else: 
        #   d['image'] = tag_img[0].get_attribute("src")  
 
        
    except NoSuchElementException:
        pass
    
    return d


In [ ]:
# call above func
get_product_info_single(product_item)

{'image': 'https://salt.tikicdn.com/cache/280x280/ts/product/b7/94/b3/3d9fab684dbad6604e398a93b38383de.jpg',
 'name': 'Điện Thoại iPhone 11 64GB  - Hàng  Chính Hãng',
 'price': '14.450.000 ₫',
 'product_url': 'https://tiki.vn/dien-thoai-iphone-11-64gb-hang-chinh-hang-p32033717.html'}

#Function to scrape info of all products from a Page URL

In [ ]:
# Function to scrape all products from a page
def get_product_info_from_page(page_url):
    """ Extract info from all products of a specfic page_url on Tiki website
        Args:
            page_url: (string) url of the page to scrape
        Returns:
            data: list of dictionary of products info. If no products shown, return empty list.
    """
    global DRIVER

    data = []
    DRIVER.get(page_url) # Use the driver to get info from the product page
    time.sleep(5) ## Must have the sleep function
    
    products_all = DRIVER.find_elements_by_class_name("product-item")
    print(f'Found {len(products_all)} products')

    for product in products_all:
      #Look through the product and get the data
      
      each_dict = get_product_info_single(product)
      data.append(each_dict)


    return data

In [ ]:
# call above func
et_product_info_from_page(MAIN_CATEGORIES[0].get("URL"))

Found 63 products


[{'image': 'https://salt.tikicdn.com/cache/280x280/ts/product/dc/6d/b9/ead3173c9800f1b65cb8d62a73ede8e5.jpg',
  'name': 'Điện Thoại Samsung Galaxy M31 (6GB/128GB) - Hàng Chính Hãng',
  'price': '4.890.000 ₫',
  'product_url': 'https://tka.tiki.vn/pixel?data=djAwMZ6DleGWYjfwW2XSaEFvmjMlhgBhfNAFyM_IL9wBaKPvJDLlQZpKlfYqKmKfNhFSC56BCvUJl9pLlHx3qGQUKFGk8HuP0tfTB5wyPvXGGgMl2bWGPh_X1Y2bbHO_bNJlyf-YcmwkYB6Lo-m_ZSSJrXi_VPlJfrsu3o-sdVpTDZm_gz1d5Thmk7Hlp0mP8xjYIhsH1h9KwvfHrm_5kPziicOYAvmFz5VRhcHtf-wFCF1l5qtDbXBl54sOM-N6_BDUNHdZHkhnXuEKZfRCa54yDq5klw3FyexQhxkRAMvsqF7nzPeibqyNDIPmhcQulG-SDpEfiRSdkqdSQkp5b0tirKq75PvbMkZ25iSfAod2HiXPXWDS8lUQZfVfqa_EMu76PUIxdKdweGTmx-1UIVDkFk2qvqqZXpdMmnM2zouvqkEpDI8ll5Au_btmdkpQdrWLR2uiY7tBQfG13ALcT5t1IxhJhloqPBH63J9qvFeeHeIvzewyLpB9qhkRYjCz6N9nXrcr7QKFkA0n9-bDyPVLXvrsRF0paOYxbKUdP6nas72-rkyrJJeXNmTYFu1e4Nb0QS0kxVplfHPGoFRse0f3X9Nebdcgy06Bv_cNRvmDlS5DhGQPI8HdRDliO1cREm_IKC56aFzfJ_lXWlBcIBOq32npbvUnKqErFLxXR3LgVZ5lmRX1ETIZZq2cKfScrSJfEjnrMwCGDRhLlCO6ZRmxUjv-FGUKy__O1L

In [ ]:
# get all the urls
list_url = []
for i in MAIN_CATEGORIES:
  url= i.get("URL")
  list_url.append(url)
  #print(url)
list_url

['https://tiki.vn/dien-thoai-may-tinh-bang/c1789?src=c.1789.hamburger_menu_fly_out_banner',
 'https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner',
 'https://tiki.vn/thiet-bi-kts-phu-kien-so/c1815?src=c.1815.hamburger_menu_fly_out_banner',
 'https://tiki.vn/laptop-may-vi-tinh/c1846?src=c.1846.hamburger_menu_fly_out_banner',
 'https://tiki.vn/may-anh/c1801?src=c.1801.hamburger_menu_fly_out_banner',
 'https://tiki.vn/dien-gia-dung/c1882?src=c.1882.hamburger_menu_fly_out_banner',
 'https://tiki.vn/nha-cua-doi-song/c1883?src=c.1883.hamburger_menu_fly_out_banner',
 'https://tiki.vn/bach-hoa-online/c4384?src=c.4384.hamburger_menu_fly_out_banner',
 'https://tiki.vn/me-va-be/c2549?src=c.2549.hamburger_menu_fly_out_banner',
 'https://tiki.vn/lam-dep-suc-khoe/c1520?src=c.1520.hamburger_menu_fly_out_banner',
 'https://tiki.vn/the-thao/c1975?src=c.1975.hamburger_menu_fly_out_banner',
 'https://tiki.vn/o-to-xe-may-xe-dap/c8594?src=c.8594.hamburger_menu_fly_out_ban

In [ ]:
######################
### START SCRAPING ###
######################

num_max_page = 2
main_cat_url = MAIN_CATEGORIES[0]['URL']

start_driver(force_restart=True)

#CODE TO GET DATA
total_data = []
for x in range(1):               #len(MAIN_CATEGORIES)):
  main_cat_url = MAIN_CATEGORIES[x].get("URL")
  for i in range(num_max_page):
    #https://tiki.vn/tivi-thiet-bi-nghe-nhin/c4221?src=c.4221.hamburger_menu_fly_out_banner
    page_link = main_cat_url.split("?")[0]+"?page"+str(i+1)+"&"+MAIN_CATEGORIES[0].get("URL").split("?")[1]
    data= get_product_info_from_page(page_link)
    total_data.extend(data)
    # for item in data:
    #   print(item)  


close_driver()

#SAVE DATA TO CSV FILE
df = pd.DataFrame(total_data, columns=total_data[0].keys())
df.to_csv('tiki_products.csv',encoding='utf-8-sig')


Found 63 products
Found 63 products


In [ ]:
total_data

[{'image': 'https://salt.tikicdn.com/cache/280x280/ts/product/50/f1/b3/7ff75fd017f968c3faf3cdfded49bd26.jpg',
  'name': ' Máy Tính Bảng HUAWEI MatePad T 10 | Hiệu Suất Mượt Mà | Chế Độ Bảo Vệ Mắt | Âm Thanh Nổi Sống Động | Hàng Chính Hãng ',
  'price': '3.990.000 ₫',
  'product_url': 'https://tka.tiki.vn/pixel?data=djAwMesLGhoS9QJG3vltyrm1VQy37fLz1cB5AOj-idTnJ43pzRD7xo27pM6W0OUda178OnTDvEjL3nCzVsLz-hPSi8Xge95YRgiZI-YqltJgnzFkshEphodJ8EUvbJOet14OWlkUYh4E0QkOuFELXvOQ5rwexyLxiBvNiO5-n8lAfLoCzZuPzVdUJsL5diP2fE3z20Vz3QyOAzUAvlkdg8ZgsvH5H0rtBpvl1dhht6srUwXHFY5BVNr8ZaEO0z--hIvfej46Z-SANWB-_FLk0BbzHPIQ6nODN0-2IXB8hZoJh_Wh99YtbtS4L6RlFIJQyHOIqSZPXza78vVXMExHcNqyBUJC5x9yQy87wjgQd44KFAaz3DCLTR-Wys1B-7HVQpVMzEEruza6U85L9oceEIn0AxUUrFzJ9ESqN9CbxryWyUbwau201LCTSwqNiuQaG3duEwyXNGUIYvkhUX_qRBEyXGKfSN11Lo1Sp1gAaYG1U1ImEGv9OepKeTLzd_DoyR3fCsg4oz0oNvU0b4O77CF9gIrCOLQYDjAnYE6ujghtyGHsH69JLD4hPt3iUJeIU_gXRYSXvwtvNJiNmOcjHRz6_w6dKuQThCAlEVcTuLviOdERQ5ozdeb4dyL-KEbBX63r0YLLed-Yasixy4WJBLFpy3KvkgYJdegvGiTi7xd

In [ ]:
# print file
df

,name,price,product_url,image
0,Máy Tính Bảng HUAWEI MatePad T 10 | Hiệu Suất...,3.990.000 ₫,https://tka.tiki.vn/pixel?data=djAwMe0xau1XIlI...,https://salt.tikicdn.com/cache/280x280/ts/prod...
1,Điện Thoại iPhone 11 64GB - Hàng Chính Hãng,14.450.000 ₫,https://tiki.vn/dien-thoai-iphone-11-64gb-hang...,https://salt.tikicdn.com/cache/280x280/ts/prod...
2,Điện thoại Vivo Y12s - Hàng Chính Hãng,2.930.000 ₫,https://tiki.vn/dien-thoai-vivo-y12s-hang-chin...,https://salt.tikicdn.com/cache/280x280/ts/prod...
3,Điện Thoại Samsung Galaxy M11 (3GB/32GB) - Hàn...,2.590.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-m11-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
4,Điện Thoại Samsung Galaxy M31 (6GB/128GB) - Hà...,4.890.000 ₫,https://tka.tiki.vn/pixel?data=djAwMamuC-HDIm6...,https://salt.tikicdn.com/cache/280x280/ts/prod...
...,...,...,...,...
121,Điện Thoại Nokia 105 Dual Sim (2019) - Hàng Ch...,360.000 ₫,https://tiki.vn/dien-thoai-nokia-105-dual-sim-...,https://salt.tikicdn.com/cache/280x280/ts/prod...
122,Điện Thoại Xiaomi Redmi Note 10 Pro (8GB/128GB...,6.599.000 ₫,https://tiki.vn/dien-thoai-xiaomi-redmi-note-1...,https://salt.tikicdn.com/cache/280x280/ts/prod...
123,Điện Thoại Samsung Galaxy Note 20 Ultra (8GB/2...,17.319.000 ₫,https://tiki.vn/dien-thoai-samsung-galaxy-note...,https://salt.tikicdn.com/cache/280x280/media/c...
124,iPad Pro M1 11 inch (2021) 128GB Wifi - Hàng C...,20.990.000 ₫,https://tiki.vn/ipad-pro-m1-11-inch-2021-128gb...,https://salt.tikicdn.com/cache/280x280/ts/prod...
